In [1]:
import os
import torch
import numpy as np
import pandas as pd
import torchvision as tv
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import (
    LearningRateMonitor,
    ModelCheckpoint,
    EarlyStopping,
)
from src.utils.helpers import load_config
from src.training.dataset import ImageDataModule
from src.models.classification_model import ImageClassifier

In [2]:
real_filepaths = os.listdir("src/data/sampled/train/real")
fake_filepaths = os.listdir("src/data/sampled/train/real")

real_count, fake_count = len(real_filepaths), len(fake_filepaths)
print(f"REAL COUNT: {real_count} ({round(real_count/(fake_count+real_count) * 100, 1)}%)")
print(f"FAKE COUNT: {fake_count} ({round(fake_count/(fake_count+real_count) * 100, 1)}%)")

REAL COUNT: 16800 (50.0%)
FAKE COUNT: 16800 (50.0%)


In [3]:
torch.set_float32_matmul_precision("high")
config = load_config("src/configs/sampled_convnext_tiny.yaml")
pl.seed_everything(config.get("seed"))

csv_logger = CSVLogger(
    save_dir=config["logging"].get("logs_dir", "logs"),
    name=config.get("experiment_name"),
)

Seed set to 7643


In [4]:
callbacks = []
data_module = ImageDataModule(config)

# Initialize model:
net = ImageClassifier(config)

# Enable learning rate monitoring hook:
if config["training"].get("lr_monitoring"):
    monitor_lr = LearningRateMonitor(logging_interval="epoch")
    callbacks.append(monitor_lr)

# Enable early stopping hook:
estop = config["training"].get("early_stopping", {})
if estop.get("enabled"):
    early_stopping = EarlyStopping(
        monitor=estop.get("monitor", "val_loss"),
        patience=estop.get("patience", 5),
        mode=estop.get("mode", "min"),
        verbose=True,
    )
    callbacks.append(early_stopping)

# Enable model checkpointing hook:
checkpoint = ModelCheckpoint(
    dirpath=config["logging"].get("checkpoint_dir", "checkpoints"),
    filename="{epoch:02d}-{val_acc:.2f}",
    save_top_k=1,
    monitor="val_acc",
    mode="max",
    save_last=True,
)
callbacks.append(checkpoint)

In [5]:
# Configure training session:
trainer = pl.Trainer(
    min_epochs=config["training"].get("min_epochs"),
    max_epochs=config["training"].get("max_epochs", -1),
    logger=csv_logger,
    enable_checkpointing=True,
    enable_progress_bar=True,
    enable_model_summary=True,
    devices="auto",
    accelerator="auto",
    log_every_n_steps=5,
    callbacks=callbacks,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
# Begin training:
trainer.fit(
    model=net,
    datamodule=data_module,
)

[NOTICE] Found 50 corrupted files in src/data/sampled/train.
[NOTICE] Found 5 corrupted files in src/data/sampled/validation.


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:751: Checkpoint directory /teamspace/studios/this_studio/Reveal-AI/src/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | loss_func | CrossEntropyLoss | 0      | train
1 | backbone  | ConvNeXt         | 27.8 M | train
-------------------------------------------------------
27.8 M    Trainable params
0         Non-trainable params
27.8 M    Total params
111.287   Total estimated model params size (MB)
250       Modules in train mode
0         Modules in eval mode


[NOTICE] Found 13 corrupted files in src/data/sampled/test.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved by 0.029 >= min_delta = 0.0. New best score: 0.062


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [7]:
# Test best model:
trainer.test(
    model=net,
    datamodule=data_module,
    ckpt_path="src/checkpoints/sample-convnext-tiny-last.ckpt",
)

[NOTICE] Found 50 corrupted files in src/data/sampled/train.
[NOTICE] Found 5 corrupted files in src/data/sampled/validation.


Restoring states from the checkpoint path at src/checkpoints/sample-convnext-tiny-last.ckpt


[NOTICE] Found 13 corrupted files in src/data/sampled/test.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at src/checkpoints/sample-convnext-tiny-last.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9835814833641052     │
│         test_loss         │   0.058173228055238724    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.058173228055238724, 'test_acc': 0.9835814833641052}]

In [8]:
# Review training metrics:
training_metrics_df = pd.read_csv(
    "src/logs/sampled_convnext_tiny/version_1/metrics.csv"
)
training_metrics_df

,epoch,lr-AdamW,step,test_acc,test_loss,train_acc,train_loss,val_acc,val_loss
0,NaN,0.00001,0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,349,NaN,NaN,NaN,NaN,0.963586,0.090572
2,0.0,NaN,349,NaN,NaN,0.913651,0.199609,NaN,NaN
3,NaN,0.00001,350,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,699,NaN,NaN,NaN,NaN,0.967616,0.092527
5,1.0,NaN,699,NaN,NaN,0.985872,0.038928,NaN,NaN
6,NaN,0.00001,700,NaN,NaN,NaN,NaN,NaN,NaN
7,2.0,NaN,1049,NaN,NaN,NaN,NaN,0.981654,0.061707
8,2.0,NaN,1049,NaN,NaN,0.995976,0.011683,NaN,NaN
9,NaN,0.00001,1050,NaN,NaN,NaN,NaN,NaN,NaN
